In [1]:
import requests
import io
import os
import json
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# EXTENSIVE CALL DATA
url = 'API_ENDPOINT_HERE/v2/calls/extensive'
# REPLACE 'API_ENDPOINT_HERE' WITH YOUR GONG API ENDPOINT
from_date = "2023-06-12T02:30:00-05:00"
to_date = "2023-06-13T02:30:00-05:00"

params = {
    "filter": {
    "fromDateTime": from_date,
     "toDateTime": to_date
    },
    "contentSelector": {
      "context": "None",
      "exposedFields": {
        "parties": True,
        "content": {
          "structure": False,
          "topics": False,
          "trackers": False,
          "trackerOccurrences": False,
          "pointsOfInterest": False
        },
        "interaction": {
          "speakers": True,
          "video": True,
          "personInteractionStats": True,
          "questions": True
        },
        "collaboration": {
          "publicComments": True
        },
        "media": True
      }
    }
}

# AUTH: ENCODE YOUR GONG API_KEY AND API_SECRET CONCATENATED WITH A COLON 
# (E.G., 'a3n5kj2j:89472lkfjkjhf9wik') AS BASE64 AND REPLACE "YOUR_API_KEY_HERE" 
# WITH THE RESULTING BASE64 STRING
headers = {
    'Authorization': 'Basic YOUR_API_KEY_HERE',
    "Content-Type": "application/json"
}

params = json.dumps(params)

response = requests.post(url, data=params, headers=headers)
print(response)
print(response.json())
calls = response.json()['calls']
print(calls)


In [ ]:

# Create the directory if it doesn't exist
if not os.path.exists('/content/drive/MyDrive/gong-videos'):
    os.makedirs('/content/drive/MyDrive/gong-videos')

# Loop through the calls and download the videos
for call in calls:
  print(call)

  if 'media' in call and 'audioUrl' in call['media']:
    # Get the audio URL
    audio_url = call['media']['audioUrl']

    # Download the video
    response = requests.get(audio_url)
    file_bytes = response.content

    # Save the video to Google Drive
    call_title = call['metaData']['title']
    call_date = call['metaData']['started'].split('T')[0]
    call_name = call_date + call_title
    call_name = call_name.replace('/','-')
    file_name = f'{call_name}.mp3'
    print('file name: ', file_name)
    with open('/content/drive/MyDrive/gong-videos/{}'.format(file_name), 'wb') as f:
        f.write(file_bytes)

  # Get the video URL
  if 'media' in call and 'videoUrl' in call['media']:
      video_url = call['media']['videoUrl']

      # Download the video
      response = requests.get(video_url)
      file_bytes = response.content

      # Save the video to Google Drive
      call_title = call['metaData']['title']
      call_date = call['metaData']['started'].split('T')[0]
      call_name = call_date + call_title
      call_name = call_name.replace('/','-')
      file_name = f'{call_name}.mp4'
      print('file name: ', file_name)
      with open('/content/drive/MyDrive/gong-videos/{}'.format(file_name), 'wb') as f:
          f.write(file_bytes)

print('Done!')